In [21]:
import torch
import json

In [22]:
with open('/home/remote_user/sharefile/AIoT_code/time_series_model/data/PD.json', 'r') as file:
    PDdata = json.load(file)
PDmax = PDdata['PDmax']
PDmin = PDdata['PDmin']

with open('/home/remote_user/sharefile/AIoT_code/time_series_model/data/Temp.json', 'r') as file:
    Tdata = json.load(file)
Tmax = Tdata['Tmax']
Tmin = Tdata['Tmin']

In [23]:
def temp_normalize(T):
    return ((T - Tmin)/(Tmax - Tmin))
        
def PD_normalize(PD):    
    return ((PD - PDmin) / (PDmax - PDmin))

def temp_denormalize(T):
    return ((Tmax - Tmin) * T + Tmin)
        
def PD_denormalize(PD):    
    return ((PDmax - PDmin) * PD + PDmin)

In [24]:
# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

path = '/home/remote_user/sharefile/AIoT_code/time_series_model/model/PD_Model.pkl'
loaded_model = torch.load(path)
PD_model = loaded_model['model'].to(device)
PD_model.eval()

path = '/home/remote_user/sharefile/AIoT_code/time_series_model/model/Temp_Model.pkl'
loaded_model = torch.load(path)
Temp_model = loaded_model['model'].to(device)
Temp_model.eval()

LSTM(
  (lstm): LSTM(9, 64, num_layers=3, batch_first=True)
  (linear): Linear(in_features=64, out_features=1, bias=True)
)

In [25]:
ori_data = torch.load('/home/remote_user/sharefile/AIoT_code/time_series_model/data/dataset.pt')

In [ ]:
ori_data.shape

In [36]:
ori_data[1000][0][0]

tensor(0.0476)

In [38]:
temp_denormalize(ori_data[1000][0][0])

tensor(25.2619)

In [39]:
ori_data[1000][-1][2] = 1
res = Temp_model(ori_data[0].unsqueeze(0).to(device))
temp_denormalize(res.item())

30.185998857021332

In [29]:
ori_data[1000][-1][2] = 0.7
res = Temp_model(ori_data[0].unsqueeze(0).to(device))
temp_denormalize(res.item())

30.198460817337036

In [30]:
ori_data[0][-1][2] = 0
res = Temp_model(ori_data[0].unsqueeze(0).to(device))
temp_denormalize(res.item())

30.211024403572083